In [3]:
# 整理数据的库
import numpy as np
import pandas as pd
import pyreadstat #读取sav格式文件需要
# 数据可视化的库
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题
# 数据分析的库
from scipy import stats
import scipy.stats as stats
import statsmodels.formula.api as smf # 回归分析
from scipy.stats import chi2_contingency #卡方检验
import pingouin as pg
from statsmodels.stats.multicomp import pairwise_tukeyhsd #事后检验
# 其它
import warnings
warnings.filterwarnings('ignore') #忽视警告

In [9]:
df_1,meta_1 = pyreadstat.read_sav("D:\\data\\CGSS\\CGSS2021.sav") #dataframe的值为纯数字

In [10]:
df_2,meta_2 = pyreadstat.read_sav(r"D:\data\CGSS\CGSS2021.sav",apply_value_formats=True)# dataframe的值为元信息
#注意：李克特计分方法的变量pyreadstat认定为类别型变量，这个和我们的需求是不同的

In [11]:
df_1.head()

,id,provinces,community_i,type,A00,A1,A011601,A011602,A011603,A011604,...,P22_b,P22_c,isco08_a57e,isco08_a59d,isco08_a60d,isco08_sp,isco08_f,isco08_m,weight,weight_raking
0,2504.0,北京市,1101021,1.0,2021-07-11 15:03:00,1.0,2.0,NaN,NaN,NaN,...,4.0,1.0,2342.0,5322.0,NaN,NaN,3341.0,9622.0,0.757877,0.265570
1,1187.0,北京市,1101022,1.0,2021-07-06 11:14:00,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,7230.0,NaN,1321.0,NaN,1112.0,NaN,0.416885,0.073736
2,4315.0,北京市,1101011,1.0,2021-07-20 16:05:00,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,2330.0,NaN,7521.0,6100.0,0.299796,0.023887
3,6836.0,北京市,1101154,1.0,2021-07-31 15:18:00,7.0,3.0,3.0,5.0,5.0,...,NaN,NaN,7230.0,2523.0,NaN,4110.0,8300.0,2341.0,0.781762,0.274609
4,4726.0,北京市,1101064,1.0,2021-07-23 09:22:00,4.0,1.0,2.0,6.0,7.0,...,1.0,2.0,7318.0,NaN,4110.0,NaN,7230.0,NaN,0.616473,0.215613


In [12]:
df_1.columns

Index(['id', 'provinces', 'community_i', 'type', 'A00', 'A1', 'A011601',
       'A011602', 'A011603', 'A011604',
       ...
       'P22_b', 'P22_c', 'isco08_a57e', 'isco08_a59d', 'isco08_a60d',
       'isco08_sp', 'isco08_f', 'isco08_m', 'weight', 'weight_raking'],
      dtype='object', length=700)

In [13]:
df_2

,id,provinces,community_i,type,A00,A1,A011601,A011602,A011603,A011604,...,P22_b,P22_c,isco08_a57e,isco08_a59d,isco08_a60d,isco08_sp,isco08_f,isco08_m,weight,weight_raking
0,2504.0,北京市,1101021,居委会,2021-07-11 15:03:00,1.0,子女,NaN,NaN,NaN,...,有很大影响,根本没有影响,幼教教师,居家个人护理工作人员,NaN,NaN,办公室主管,零工,0.757877,0.265570
1,1187.0,北京市,1101022,居委会,2021-07-06 11:14:00,1.0,配偶,NaN,NaN,NaN,...,NaN,NaN,机械装配和修理工,NaN,制造业经理,NaN,高级政府官员,NaN,0.416885,0.073736
2,4315.0,北京市,1101011,居委会,2021-07-20 16:05:00,1.0,配偶,NaN,NaN,NaN,...,NaN,NaN,军职人员,NaN,中学教师,NaN,木材处理工人,面向市场的熟练农业工作人员,0.299796,0.023887
3,6836.0,北京市,1101154,居委会,2021-07-31 15:18:00,7.0,父母,父母,兄弟姐妹,兄弟姐妹,...,NaN,NaN,机械装配和修理工,计算机网络专业人员,NaN,一般办公室工作人员,驾驶和移动设备操作员,小学教师,0.781762,0.274609
4,4726.0,北京市,1101064,居委会,2021-07-23 09:22:00,4.0,配偶,子女,女婿/儿媳,祖父母/外祖父母,...,根本没有影响,有一点影响,纺织、皮革及相关材料的手工艺工人,NaN,一般办公室工作人员,NaN,机械装配和修理工,NaN,0.616473,0.215613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8143,6215.0,宁夏回族自治区,6403241,居委会,2021-07-28 11:31:00,3.0,父母,父母,兄弟姐妹,NaN,...,有一点影响,根本没有影响,不知道,NaN,NaN,NaN,未分类的简单劳动工作人员,农业、林业、牧业、渔业熟练工作人员,1.587054,2.703331
8144,7068.0,宁夏回族自治区,6403243,村委会,2021-08-03 10:31:00,6.0,配偶,子女,女婿/儿媳,孙子（女）/外孙子（女）,...,有一点影响,有一些影响,不适用,NaN,NaN,NaN,NaN,面向市场的熟练农业工作人员,0.292696,0.133989
8145,6625.0,宁夏回族自治区,6403241,居委会,2021-07-29 12:56:00,3.0,配偶,子女,子女,NaN,...,NaN,NaN,不知道,不知道,NaN,不知道,不知道,不知道,1.500252,0.897839
8146,6659.0,宁夏回族自治区,6403241,居委会,2021-07-29 11:24:00,3.0,配偶,子女,子女,NaN,...,NaN,NaN,护理专业人员,护理和助产专业技术人员,NaN,一般办公室工作人员,泥水匠,NaN,0.767249,0.221671


# 数据清洗

定义函数，把“无法选择=98”、“无法选择=99”等结果变成空值

In [15]:
def drop_98(series):
    if series == 98:
        return ""
    else:
        pass

def drop_99(series):
    if series == 99:
        return ""
    else:
        pass

def drop_98(series):
    if series == 98:
        return ""
    else:
        pass


